In [1]:
import numpy as np
from cvxpy import *

# random seed
np.random.seed(0)

# Problem data.
n = 100
i = 20
y = np.random.rand(n)
# A = np.random.rand(n, i)  # normal
A = np.random.rand(n, i)  # in this order to test random numbers

# Construct the problem.
x = Variable(n)
lmbd = Variable(i)
objective = Minimize(sum_squares(x - y))
constraints = [x == A*lmbd,
               lmbd >= np.zeros(i),
               sum_entries(lmbd) == 1]

prob = Problem(objective, constraints)
result = prob.solve(verbose=True)



ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -2.654e-17  -1.123e-02  +7e+01  4e-01  6e-02  1e+00  3e+00    ---    ---    1  1  - |  -  - 
 1  -7.280e-01  -7.443e-01  +5e+00  5e-02  4e-03  5e-02  2e-01  0.9271  3e-04   1  1  2 |  0  0
 2  +1.946e+00  +2.137e+00  +4e+00  1e-01  7e-03  3e-01  2e-01  0.6909  6e-01   2  2  2 |  0  0
 3  +3.495e+00  +3.516e+00  +6e-01  2e-02  8e-04  4e-02  3e-02  0.8353  7e-03   2  2  2 |  0  0
 4  +3.809e+00  +3.902e+00  +5e-01  5e-02  1e-03  1e-01  3e-02  0.4028  7e-01   2  2  3 |  0  0
 5  +4.850e+00  +4.920e+00  +3e-01  3e-02  8e-04  9e-02  1e-02  0.4403  8e-02   2  2  2 |  0  0
 6  +3.923e+00  +4.020e+00  +3e-01  3e-02  8e-04  1e-01  1e-02  0.4106  7e-01   2  2  2 |  0  0
 7  +4.484e+00  +4.560e+00  +2e-01  2e-02  5e-04  9e-02  1e-02  0.5762  6e-01   2  2  2 |  0  0
 8  +4.752e+00  +4.913e+00  +1e-01  3e-02  6e-

In [3]:
num_features = 3
# should add bias dimension?
# what is lambda in the existing implementation. do we need it?

def solver(D, q=2, d=50, num_states=750, num_actions=25):
    """
    Maximum Margin Planning Algorithm
    
    solve a QP
    
    min w.r.t. W, Z
    (1/2) ||w||^2 + (gamma / n) sum
    
    
    """
    # number of data points
    n = D.shape[0]
    # dimension of features
    d = 50
    w = Variable(d)
    z = Variable(d)    
    # hyperparameters
    # beta strictly positive
    beta = Parameter(sign='positive')
    beta.value = np.ones((d, 1))
    # gamma non negative
    gamma = Parameter(sign='positive', value=3.0)
    
    # define a loss matrix here
    # we will need to keep track of indices
    L = Parameter(sign='positive')
    # can add a custom mask on L
    # the idea is to 
    L.value = np.reciprocal(expert_historical_sa)
    # constraints
    if q == 2 or q == 1:
        # l2 slack variable
        objective = Minimize(0.5 * norm(w, q) + (1/n) * gamma * sum_entries(beta*z))
    else:
        print('q should be 1 or 2')
        return
    # need to take average mu across a trajectory
    constraints = [w.T * mu >= max_entries(w.T * mu) + L]
    prob = Problem(objective, constraints)
    prob.solve()
    
    print('optimal weights', w.value)
    print('optimal slack', z.value)
    